## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-15-43-08 +0000,bbc,'We've lost everything': Australia wildfire ri...,https://www.bbc.com/news/videos/cr4de0x2xqvo?a...
1,2025-12-06-15-32-48 +0000,bbc,Scottish Conservative peer defects to Reform UK,https://www.bbc.com/news/articles/c5yq9yr31mgo...
2,2025-12-06-15-30-26 +0000,nyt,Survivors of the Deadly Hong Kong Fire Are in ...,https://www.nytimes.com/2025/12/06/world/asia/...
3,2025-12-06-15-22-55 +0000,nyt,Trump’s Security Doctrine Leaves Europe at a S...,https://www.nytimes.com/2025/12/06/world/europ...
4,2025-12-06-15-08-31 +0000,bbc,Bethlehem Christmas tree lights up for first t...,https://www.bbc.com/news/articles/ce86z84e947o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,45
176,netflix,18
97,new,16
178,warner,16
377,court,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
217,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,99
250,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...,95
202,2025-12-05-19-51-16 +0000,cbc,U.S. Supreme Court to decide whether Trump's b...,https://www.cbc.ca/news/world/supreme-court-tr...,89
208,2025-12-05-19-33-39 +0000,nypost,Supreme Court to decide whether or not Trump’s...,https://nypost.com/2025/12/05/us-news/supreme-...,88
159,2025-12-05-22-05-51 +0000,nyt,Supreme Court Agrees to Review Trump Order Res...,https://www.nytimes.com/2025/12/05/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
217,99,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...
254,74,2025-12-05-17-40-40 +0000,nypost,Here’s everything you need to know about the N...,https://nypost.com/2025/12/05/business/heres-e...
268,62,2025-12-05-16-51-04 +0000,nypost,DC pipe-bomb suspect Brian Cole Jr. appears in...,https://nypost.com/2025/12/05/us-news/accused-...
269,45,2025-12-05-16-44-00 +0000,wsj,New York Times Escalates Battle Against Perple...,https://www.wsj.com/business/media/new-york-ti...
101,41,2025-12-06-02-00-00 +0000,wsj,Takeaways from Robert F. Kennedy Jr.’s vaccine...,https://www.wsj.com/politics/skeptics-in-the-s...
250,38,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...
18,38,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
19,37,2025-12-06-14-31-08 +0000,cbc,Russia unleashes massive drone and missile att...,https://www.cbc.ca/news/world/russia-ukraine-a...
35,35,2025-12-06-12-52-57 +0000,nypost,National Guard shooting suspect’s close milita...,https://nypost.com/2025/12/06/us-news/national...
109,34,2025-12-06-01-02-00 +0000,wsj,Two survivors of a Sept. 2 lethal U.S. strike ...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
